In [2]:
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import sys
import numpy as np

In [3]:
onehot = OneHotEncoder(sparse=False)
X, Y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = scale(X)
Y = onehot.fit_transform(Y.reshape(-1, 1))
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

/home/lucas/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:
number_of_classes = 10
y_pred = None
with tf.Session() as sess:
    with tf.variable_scope('LOGISTIC_REGRESSION', reuse=tf.AUTO_REUSE):
        old_loss = None
        x = tf.placeholder(tf.float64, shape=(None, 784), name='x')
        y = tf.placeholder(tf.float64, shape=(None, number_of_classes), name='y')

        w = tf.get_variable('w', shape=(784, number_of_classes), dtype=tf.float64, initializer=tf.random_normal_initializer())
        b = tf.get_variable('b', shape=(number_of_classes), dtype=tf.float64, initializer=tf.random_normal_initializer())

        reg = tf.matmul(x, w) + b
        softmax = tf.round(tf.nn.softmax(reg))
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=reg, labels=y))

        optimizer = tf.train.AdamOptimizer(0.03).minimize(loss)
        sess.run(tf.global_variables_initializer())
        iterations = 100
        for i in range(iterations):
            result, loss_value, _ = sess.run([softmax, loss, optimizer], feed_dict={x:x_train, y:y_train})

            if old_loss:
                if loss_value > old_loss:
                    break

            old_loss = loss_value
            
            sys.stdout.write(f"\r {i+1} from {iterations} iteration Loss: " + str(loss_value))
            sys.stdout.flush()

        y_pred = sess.run(softmax, feed_dict={x:x_test})
    

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

 100 from 100 iteration Loss: 1.179892518566791

In [5]:
print("Accuracy Score:", accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

Accuracy Score: 0.8778095238095238
